In [19]:
import sys
sys.path.append("..")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Feature Engineering for Cohort Profitability Prediction

This notebook creates features for predicting ROI at horizon H using only information available up to decision time t.

## Key Parameters
- **Decision Time (t)**: 90 days after cohort creation (parametrized for easy modification)
- **Horizon (H)**: Based on EDA findings, we use the full observation period for final ROI calculation
- **Feature Scope**: Only information available at or before time t is used

## Feature Categories
1. **Loan-Level Features**: Individual loan characteristics and early behavior signals
2. **Cohort-Level Features**: Portfolio composition and risk distribution metrics

In [20]:
# Parameters - easily configurable
from src.config import DECISION_TIME_DAYS, TIME_HORIZON_DAYS, DATABASE_PATH

print(f"Decision time set to: {DECISION_TIME_DAYS} days after cohort creation")
print(f"Time horizon set to: {TIME_HORIZON_DAYS} days for target variable")

Decision time set to: 400 days after cohort creation
Time horizon set to: 600 days for target variable


## Data Loading and Preparation

In [21]:
from src.dataset.data_manipulation import load_data

# Load all data
allowlist, loans, repayments, loans_and_cohort, repayments_and_loans = load_data(
    # DATABASE_PATH, remove_loans_with_errors=
    DATABASE_PATH, remove_loans_with_errors=False
)

## Feature Engineering Functions

We'll import feature engineering functions from a dedicated module to keep the notebook clean and functions reusable.

In [22]:
from src.features import (
    create_loan_level_features,
    create_cohort_level_features,
    save_features_to_database
)

## 1. Loan-Level Features

### Loan Characteristics
- Loan amount (raw and log-transformed)
- Annual interest rate
- Loan size decile within cohort

### Temporal Features
- Time since loan issuance at decision time t
- Time between allowlist date and loan creation

### Interaction Terms
- Loan amount × interest rate
- Loan ROI at 30/60/90 days

### Early Repayment Behavior
- Days to first repayment
- Repayment velocity (30/60/90 days)
- Repayment consistency metrics

### Repayment Quality Indicators
- Average repayment amount relative to loan size
- Repayment acceleration/deceleration trends

### Billing Payment Indicators
- Time in billing process
- Is in normal repayment process (boolean)

In [23]:
loan_features_df = create_loan_level_features(
    loans_and_cohort,
    repayments_and_loans,
    decision_time_days=DECISION_TIME_DAYS,
    time_horizon_days=TIME_HORIZON_DAYS,
)
loan_features_df

Creating loan-level features with decision time t=400 days...
Base features dataset: 36287 unique loans
Creating repayment behavior features...


c:\Users\vitor\git_repos\case-ds-cloud-walk\notebooks\..\src\features\loan_features.py:166: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  repayments_filtered.groupby("loan_id").apply(calc_consistency).reset_index()


Final loan features dataset: 36287 loans with 30 features


,loan_id,user_id,created_at,updated_at,annual_interest,loan_amount,status_at_decision_time,batch,allowlisted_date,batch_letter,...,repayment_velocity_266d,loan_roi_266d,repayment_velocity_399d,loan_roi_399d,days_to_first_repayment,num_repayments,total_repaid_amount,repayment_consistency_cv,avg_repayment_relative,repayment_at_H
0,0000634b4de08f4d798a4546bd104aa5d3e43af416bd48...,e00cc67f993040157c1a5d15b35d8b6182e567c405fff9...,2024-03-11,2025-01-07 22:47:43.375,2.4,4000.0,debt_collection,9a65c2254d6d2b240f353b95df7061928c7a9869417325...,2023-12-19,F,...,1.048087,-0.952050,0.606962,-0.952050,1.0,24,191.800000,1.450956,0.001998,191.800000
1,000084327034f5aea172294e82f81cc7f4c24162a075bc...,250761407286bebafb435d00b7568e7e476de772abfbf7...,2023-03-30,2023-05-24 01:04:32.107,2.4,3250.0,debt_collection,5bcbc3d39978a3ff54a2671faf77e3e43c798faf53e98f...,2022-09-09,E,...,9.217031,-0.818495,6.498223,-0.606108,1.0,16,1280.150000,0.932280,0.024618,1280.150000
2,00022546590af574f1785cb5e4c17bb1898de7bce40977...,1532d16402c104350db26e145d562e7b9ef392e16e9c99...,2023-12-07,2025-01-07 22:40:28.233,3.2,500.0,debt_collection,4398a3e49d78f4b1b816ced315f34a5da5e830b1f53640...,2023-12-05,D,...,0.181818,-0.904000,0.120907,-0.904000,9.0,1,48.000000,0.000000,0.096000,48.000000
3,000402c18c2931e31e9cd68b5a01d1389337e55572859a...,35bd33ed5eb7a85c88c2b1baf1ec368adc994b9bdc9f5e...,2024-08-12,2024-08-12 15:14:57.424,3.4,50.0,repaid,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,...,0.369412,0.004800,0.186766,0.004800,0.0,1,50.240000,0.000000,1.004800,50.240000
4,000dca06cc48943ca84d7516f817709f2b7768468a9a02...,445a2b25d6692ec55caf314c6bc998c517ea9022c65735...,2024-06-01,2024-06-03 12:02:32.785,3.4,50.0,repaid,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,...,0.242500,0.008800,0.147918,0.008800,1.0,2,50.440000,0.984140,0.504400,50.440000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36282,fffa7d663d32bfa90ca35a874ef5b2a842595b7627dd39...,627575c514eec900ec0ac9f1780fb41c92708b3889b58e...,2024-07-27,2024-07-31 23:03:34.911,3.4,50.0,repaid,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,...,0.335263,0.019200,0.178807,0.019200,2.0,3,50.960000,0.226884,0.339733,50.960000
36283,fffb5b06cc5ef2d4fd3d9321bc797d95b0bdb75ac77215...,4f1efc1e1af62ccdbc89ac564d33c22ed3021c6d3be748...,2024-04-11,2024-04-12 15:31:41.127,3.4,50.0,repaid,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,...,0.194286,0.006400,0.128367,0.006400,1.0,1,50.320000,0.000000,1.006400,50.320000
36284,fffccf877a6b7745194286d6683b55d9d69ce2a800e64f...,ef9fa866ffbbd757283c5ade094cef617518b80cdb7bbc...,2023-01-23,2023-03-07 14:20:01.960,2.4,3000.0,repaid,1d83f7f96a6a3a06b30bc683b94a428225fe072e60959f...,2022-08-29,B,...,26.763934,0.061636,12.638525,0.061636,9.0,12,3184.908196,1.096383,0.088470,3184.908196
36285,fffcffd247c02bfc1d42974623254a88eeee39b46dbd6b...,b1862108e0314a10a21ad8b4ea4193016fce49014a3868...,2024-05-29,2024-06-03 21:12:11.047,3.4,100.0,repaid,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,...,0.484123,0.021500,0.296948,0.021500,1.0,2,102.150000,0.608419,0.510750,102.150000


In [24]:
loan_features_df[loan_features_df.repayment_at_H.isnull()]

,loan_id,user_id,created_at,updated_at,annual_interest,loan_amount,status_at_decision_time,batch,allowlisted_date,batch_letter,...,repayment_velocity_266d,loan_roi_266d,repayment_velocity_399d,loan_roi_399d,days_to_first_repayment,num_repayments,total_repaid_amount,repayment_consistency_cv,avg_repayment_relative,repayment_at_H
241,01be86d12911c4eca7702c91a97c3a35a84dc3f655803a...,afe9269ee11b6f04f02f2d47a4673e7531995760f003e3...,2024-12-13,2025-01-10 22:31:23.851,3.4,50.0,debt_collection,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,...,0.0,-1.0,0.0,-1.0,NaN,0,0.0,NaN,0.0,NaN
245,01c101ec6059c04e657b292143a01e1571ef728be57d16...,72b8db5832cc45d74deb3f6cef3c313b56d7f4420887ac...,2024-01-25,2025-01-07 22:43:14.688,3.2,500.0,debt_collection,4398a3e49d78f4b1b816ced315f34a5da5e830b1f53640...,2023-12-05,D,...,0.0,-1.0,0.0,-1.0,NaN,0,0.0,NaN,0.0,NaN
346,027eb2e6137319789c45c5f736fb031578d84e9a7d6c05...,7dee560a19f6160bb9a46c96cc3becb98ddf50e06b1a5d...,2024-10-07,2024-10-07 16:26:40.154,3.2,600.0,executed,4398a3e49d78f4b1b816ced315f34a5da5e830b1f53640...,2023-12-05,D,...,0.0,-1.0,0.0,-1.0,NaN,0,0.0,NaN,0.0,NaN
449,03280e5e427d0061dcac018bf704fbdd4cc1706bcfe780...,ef86918bd9d19a17de0aa5dca195663192f7c67c451469...,2023-12-10,2025-01-07 22:39:42.095,3.2,500.0,debt_collection,4398a3e49d78f4b1b816ced315f34a5da5e830b1f53640...,2023-12-05,D,...,0.0,-1.0,0.0,-1.0,NaN,0,0.0,NaN,0.0,NaN
473,0359a0b0bb831c4999576d34e09ec1430d6acd8ca66007...,f409053bdc1e4f974ec698b175219f86d8e20e2e11e1a3...,2024-04-29,2025-01-07 22:44:46.922,3.4,50.0,debt_collection,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,...,0.0,-1.0,0.0,-1.0,NaN,0,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35902,fd82cb8e3ffabb01ecd21882db92f4cac8b3ca6555d19a...,b462176e47c9e22f9559f9e7d0a25d9537a749854eebd7...,2023-05-24,2023-06-15 01:13:47.164,1.7,2250.0,debt_collection,22bda9c4992b30b6b347e22c27f9e59fb9ce99f1072389...,2022-09-09,C,...,0.0,-1.0,0.0,-1.0,NaN,0,0.0,NaN,0.0,NaN
35989,fe2be747e8e245cb986450df138cde5f84bb43fe4cd4e6...,f03b2434843255e4acf62262e7cdc6ff7b5adabed7be15...,2023-12-06,2025-01-07 22:41:34.127,3.2,500.0,debt_collection,4398a3e49d78f4b1b816ced315f34a5da5e830b1f53640...,2023-12-05,D,...,0.0,-1.0,0.0,-1.0,NaN,0,0.0,NaN,0.0,NaN
36067,fea4de4c3623caaff497f9a3328a16e9024221c1c343d1...,4ea3540d3005b77a923f7b746cf8a3813d9ac2eb03110d...,2024-03-18,2025-01-07 22:48:46.706,3.2,500.0,debt_collection,4398a3e49d78f4b1b816ced315f34a5da5e830b1f53640...,2023-12-05,D,...,0.0,-1.0,0.0,-1.0,NaN,0,0.0,NaN,0.0,NaN
36231,ffa14460dd4ff0ff7648ae70b653c47e4302e529d59090...,a48cdbe43c1e22589d0ab1daa64f6bf38759c11b8e3878...,2024-02-18,2025-01-07 22:46:06.467,3.2,500.0,debt_collection,4398a3e49d78f4b1b816ced315f34a5da5e830b1f53640...,2023-12-05,D,...,0.0,-1.0,0.0,-1.0,NaN,0,0.0,NaN,0.0,NaN


In [25]:
loan_id = "4dc6209ade5525396a30910e26e006749df5f878e137cb87bb123267f970bce3"
user_id = "3487c5129cdf4c202d16febed9fa29c680e54b40859d72110ad76191d31525b7"
unique_loans = loans_and_cohort[loans_and_cohort.user_id == user_id].sort_values("updated_at")[
    "loan_id"
].unique()

loans_and_cohort[loans_and_cohort.user_id == user_id].sort_values("updated_at")

,loan_id,user_id,created_at,updated_at,annual_interest,loan_amount,status,batch,allowlisted_date,batch_letter,cohort_start,created_at_h_days,updated_at_h_days
115695,4dc6209ade5525396a30910e26e006749df5f878e137cb...,3487c5129cdf4c202d16febed9fa29c680e54b40859d72...,2024-04-05,2024-04-05 15:59:24.600,3.4,200.0,executed,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,2024-04-04,1,1
115696,4dc6209ade5525396a30910e26e006749df5f878e137cb...,3487c5129cdf4c202d16febed9fa29c680e54b40859d72...,2024-04-05,2024-04-05 15:59:24.611,3.4,200.0,executed,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,2024-04-04,1,1
115694,4dc6209ade5525396a30910e26e006749df5f878e137cb...,3487c5129cdf4c202d16febed9fa29c680e54b40859d72...,2024-04-05,2024-04-05 23:46:14.231,3.4,200.0,executed,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,2024-04-04,1,1
115693,4dc6209ade5525396a30910e26e006749df5f878e137cb...,3487c5129cdf4c202d16febed9fa29c680e54b40859d72...,2024-04-05,2024-04-06 08:33:50.718,3.4,200.0,repaid,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,2024-04-04,1,2
115698,8f5dab3bc9019dbf8b4c19a9b12d294963c4af8798dd09...,3487c5129cdf4c202d16febed9fa29c680e54b40859d72...,2024-04-06,2024-04-06 09:09:13.662,3.4,100.0,executed,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,2024-04-04,2,2
115699,8f5dab3bc9019dbf8b4c19a9b12d294963c4af8798dd09...,3487c5129cdf4c202d16febed9fa29c680e54b40859d72...,2024-04-06,2024-04-06 09:09:13.669,3.4,100.0,executed,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,2024-04-04,2,2
115697,8f5dab3bc9019dbf8b4c19a9b12d294963c4af8798dd09...,3487c5129cdf4c202d16febed9fa29c680e54b40859d72...,2024-04-06,2024-04-06 09:09:40.726,3.4,100.0,repaid,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,2024-04-04,2,2
115700,7cc32caae581a1ae76e15cb84cc74ae060e78e12230c7e...,3487c5129cdf4c202d16febed9fa29c680e54b40859d72...,2024-04-06,2024-04-06 14:21:32.197,3.4,100.0,executed,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,2024-04-04,2,2
115702,7cc32caae581a1ae76e15cb84cc74ae060e78e12230c7e...,3487c5129cdf4c202d16febed9fa29c680e54b40859d72...,2024-04-06,2024-04-06 14:21:32.206,3.4,100.0,executed,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,2024-04-04,2,2
115701,7cc32caae581a1ae76e15cb84cc74ae060e78e12230c7e...,3487c5129cdf4c202d16febed9fa29c680e54b40859d72...,2024-04-06,2024-04-06 23:53:53.566,3.4,100.0,executed,e6a25e071c60243b0c51c698db5302b54ef61338c6747a...,2024-04-04,G,2024-04-04,2,2


In [26]:
repayments_and_loans[repayments_and_loans.loan_id.isin([loan_id])]

,date,loan_id,repayment_amount,billings_amount,batch_letter,allowlisted_date,loan_amount,cohort_start,created_at,created_at_h_days,h_days,repayment_total
398227,2024-04-06,4dc6209ade5525396a30910e26e006749df5f878e137cb...,201.42,0.0,G,2024-04-04,200.0,2024-04-04,2024-04-05,1,2,201.42


In [27]:
# Check available columns in loan features
print("Loan features columns:")
[print(f"- {col}") for col in loan_features_df.columns.tolist()]


# Show unique statuses
if 'status_at_decision_time' in loan_features_df.columns:
    print(f"\nUnique statuses at decision time:")

Loan features columns:
- loan_id
- user_id
- created_at
- updated_at
- annual_interest
- loan_amount
- status_at_decision_time
- batch
- allowlisted_date
- batch_letter
- cohort_start
- created_at_h_days
- updated_at_h_days
- loan_amount_log
- loan_size_decile
- days_since_loan_issuance
- days_allowlist_to_loan
- loan_amount_x_interest
- repayment_velocity_133d
- loan_roi_133d
- repayment_velocity_266d
- loan_roi_266d
- repayment_velocity_399d
- loan_roi_399d
- days_to_first_repayment
- num_repayments
- total_repaid_amount
- repayment_consistency_cv
- avg_repayment_relative
- repayment_at_H

Unique statuses at decision time:


In [28]:
loan_features_df.describe()

c:\Users\vitor\git_repos\case-ds-cloud-walk\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\vitor\git_repos\case-ds-cloud-walk\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\vitor\git_repos\case-ds-cloud-walk\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,created_at,updated_at,annual_interest,loan_amount,allowlisted_date,cohort_start,created_at_h_days,updated_at_h_days,loan_amount_log,loan_size_decile,...,repayment_velocity_266d,loan_roi_266d,repayment_velocity_399d,loan_roi_399d,days_to_first_repayment,num_repayments,total_repaid_amount,repayment_consistency_cv,avg_repayment_relative,repayment_at_H
count,36287,36287,36287.000000,36287.000000,36287,36287,36287.000000,36287.000000,36287.000000,36287.000000,...,2.998800e+04,29988.000000,3.624000e+04,36240.000000,35621.000000,36287.000000,36287.000000,35621.000000,36287.000000,35836.000000
mean,2024-02-07 16:40:42.130790656,2024-03-14 13:59:25.876641536,2.919078,1327.808238,2023-09-22 16:53:38.342657280,2023-09-22 16:53:38.342657280,137.991016,173.271282,5.971586,3.421721,...,inf,-0.048891,inf,-0.021182,6.898599,10.714002,1260.295877,0.850236,0.424853,1369.620060
min,2022-08-30 00:00:00,2022-10-10 12:42:27.332963,1.700000,5.000000,2022-08-29 00:00:00,2022-08-29 00:00:00,0.000000,1.000000,1.791759,1.000000,...,-2.076087e-02,-1.015280,-1.205047e-02,-1.015280,-1.000000,0.000000,-3.820000,0.000000,-0.005093,-3.820000
25%,2023-08-01 00:00:00,2023-09-03 11:26:03.383500032,2.400000,50.000000,2022-09-09 00:00:00,2022-09-09 00:00:00,41.000000,69.000000,3.931826,1.000000,...,2.967533e-01,0.007800,1.905050e-01,0.009400,1.000000,1.000000,51.470000,0.000000,0.053771,51.900000
50%,2024-04-23 00:00:00,2024-05-13 19:45:08.182000128,3.200000,500.000000,2023-12-19 00:00:00,2023-12-19 00:00:00,111.000000,149.000000,6.216606,2.000000,...,1.335449e+00,0.022800,1.507621e+00,0.030600,2.000000,3.000000,401.830000,0.758728,0.263005,505.140000
75%,2024-07-21 00:00:00,2024-08-22 12:06:56.872999936,3.400000,1700.000000,2024-04-04 00:00:00,2024-04-04 00:00:00,217.000000,275.000000,7.438972,5.000000,...,9.192969e+00,0.063274,8.439011e+00,0.075640,8.000000,14.000000,1539.625000,1.238281,1.004739,1667.205000
max,2025-04-26 00:00:00,2025-05-09 21:30:56.059000,3.400000,32480.000000,2024-04-04 00:00:00,2024-04-04 00:00:00,400.000000,400.000000,10.388411,10.000000,...,inf,1.463600,inf,1.463600,376.000000,143.000000,36504.440000,7.839744,2.035470,38809.350000
std,NaN,NaN,0.597916,2397.998703,NaN,NaN,112.303020,120.336150,1.680811,2.686383,...,NaN,0.292703,NaN,0.269336,20.868885,15.776525,2404.920777,0.846775,0.405836,2539.490721


## 2. Cohort-Level Features

### Portfolio Concentration Metrics
- Gini coefficient of loan amounts
- Herfindahl-Hirschman Index (HHI)
- Loan amount percentiles (P10, P25, P50, P75, P90, P95)

### Risk Distribution Metrics
- Cohort size (number of loans)
- Value-weighted average loan amount
- Statistical measures: standard deviation, skewness, coefficient of variation

In [29]:
# Create cohort-level features
print("Creating cohort-level features...")
cohort_features_df = create_cohort_level_features(
    loans_and_cohort=loans_and_cohort,
    repayments_and_loans=repayments_and_loans,
    decision_time_days=DECISION_TIME_DAYS
)

print(f"Created {len(cohort_features_df.columns)} cohort-level features for {len(cohort_features_df)} cohorts")
print("\nFeature columns:")
for col in sorted(cohort_features_df.columns):
    print(f"  - {col}")

Creating cohort-level features...
Creating cohort-level features...
Creating loan-level features with decision time t=400 days...
Base features dataset: 36287 unique loans
Creating repayment behavior features...


c:\Users\vitor\git_repos\case-ds-cloud-walk\notebooks\..\src\features\loan_features.py:166: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  repayments_filtered.groupby("loan_id").apply(calc_consistency).reset_index()


Final loan features dataset: 36287 loans with 30 features
Final cohort features dataset: 7 cohorts with 30 features
Created 30 cohort-level features for 7 cohorts

Feature columns:
  - avg_days_allowlist_to_loan
  - avg_days_since_loan_issuance
  - avg_days_to_first_repayment
  - avg_interest_rate
  - avg_loan_amount
  - avg_loan_amount_x_interest
  - avg_repayment_consistency
  - batch_letter
  - cohort_size
  - loan_amount_cv
  - loan_amount_hhi
  - loan_amount_p25
  - loan_amount_p75
  - loan_amount_p90
  - loan_amount_skewness
  - median_days_to_first_repayment
  - median_interest_rate
  - median_loan_amount
  - pct_debt_collection
  - pct_debt_repaid
  - pct_executed
  - pct_loans_in_billing
  - pct_loans_normal_repayment
  - pct_loans_totally_repaid
  - pct_repaid
  - std_days_allowlist_to_loan
  - std_days_since_issuance
  - std_interest_rate
  - total_loan_amount
  - total_repaid_amount


c:\Users\vitor\git_repos\case-ds-cloud-walk\notebooks\..\src\features\cohort_features.py:57: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features_df.groupby("batch_letter").apply(calc_group_metrics).reset_index()


In [30]:
# Display cohort-level features
print("Cohort-level features:")
display(cohort_features_df)

Cohort-level features:


,batch_letter,cohort_size,total_loan_amount,avg_loan_amount,median_loan_amount,loan_amount_skewness,avg_interest_rate,median_interest_rate,std_interest_rate,total_repaid_amount,...,median_days_to_first_repayment,avg_repayment_consistency,pct_loans_totally_repaid,pct_loans_in_billing,pct_loans_normal_repayment,pct_executed,pct_debt_collection,pct_debt_repaid,pct_repaid,avg_loan_amount_x_interest
0,A,3183,786691.62,247.154138,50.0,6.007656,3.398743,3.4,0.015806,7.890031e+05,...,1.0,0.613675,0.915489,0.014766,0.069746,0.069746,0.014766,0.029532,0.885957,839.168554
1,B,3286,10688639.30,3252.781284,2250.0,3.219970,2.400000,2.4,0.000000,1.022088e+07,...,1.0,1.522063,0.816799,0.088253,0.094948,0.094948,0.088253,0.048691,0.768107,7806.675082
2,C,4781,15901835.50,3326.048003,2250.0,2.859534,2.025329,1.7,0.349166,1.533766e+07,...,1.0,1.589397,0.838318,0.073625,0.087011,0.087011,0.073625,0.035557,0.802761,6721.837549
3,D,4519,2269704.56,502.258146,500.0,17.602285,3.200000,3.2,0.000000,2.009649e+06,...,9.0,0.560193,0.718964,0.212215,0.068599,0.068599,0.212215,0.050454,0.668511,1607.226066
4,E,2635,8238773.55,3126.669279,2250.0,3.163195,2.040569,1.7,0.349939,7.357503e+06,...,3.0,1.436752,0.774573,0.127135,0.097533,0.097533,0.127135,0.068691,0.705882,6299.070700
5,F,3133,6648485.41,2122.082799,1000.0,4.908713,2.412512,2.4,0.099278,6.423701e+06,...,3.0,1.293676,0.889244,0.035429,0.075327,0.075327,0.035429,0.030642,0.858602,5118.474620
6,G,14750,3648047.60,247.325261,50.0,9.210035,3.399376,3.4,0.011152,3.593955e+06,...,1.0,0.403945,0.889356,0.027458,0.083119,0.083119,0.027458,0.073831,0.815525,840.143094


The cohort modeling will be implemented in the following way.

We want to build a function $f$ that will predict the ROI in function of time that will work until Horizon H, using only information available until decision time $t_d$:

$$ROI_c(t \leq H) = f(\vec{\theta}(t_d), t) $$

In this function, $\vec{\theta}(t_d)$ are the cohort features that we built in this notebook, this feature vector is only the parameter set for fitting this function, and $t$ is, in fact, the dependent variable.

So, now we need to to add in the features dataframe the ROI for each $t \leq t_d$ in order to train the model.

In [31]:
import pandas as pd
import sqlite3

with sqlite3.connect(DATABASE_PATH) as conn:
    roi_df = pd.read_sql("SELECT * FROM roi_curves", conn)

In [32]:
roi_features = roi_df[
    (1 <= roi_df["h_days"]) & (roi_df["h_days"] <= TIME_HORIZON_DAYS)
][["batch_letter", "h_days", "ROI"]]  # ROI until decision time
cohort_features_with_labels = roi_features.merge(
    cohort_features_df, on=["batch_letter"], how="left"
)
cohort_features_with_labels

,batch_letter,h_days,ROI,cohort_size,total_loan_amount,avg_loan_amount,median_loan_amount,loan_amount_skewness,avg_interest_rate,median_interest_rate,...,median_days_to_first_repayment,avg_repayment_consistency,pct_loans_totally_repaid,pct_loans_in_billing,pct_loans_normal_repayment,pct_executed,pct_debt_collection,pct_debt_repaid,pct_repaid,avg_loan_amount_x_interest
0,A,1,-0.846597,3183,786691.62,247.154138,50.0,6.007656,3.398743,3.4,...,1.0,0.613675,0.915489,0.014766,0.069746,0.069746,0.014766,0.029532,0.885957,839.168554
1,A,2,-0.618791,3183,786691.62,247.154138,50.0,6.007656,3.398743,3.4,...,1.0,0.613675,0.915489,0.014766,0.069746,0.069746,0.014766,0.029532,0.885957,839.168554
2,A,3,-0.640994,3183,786691.62,247.154138,50.0,6.007656,3.398743,3.4,...,1.0,0.613675,0.915489,0.014766,0.069746,0.069746,0.014766,0.029532,0.885957,839.168554
3,A,4,-0.411533,3183,786691.62,247.154138,50.0,6.007656,3.398743,3.4,...,1.0,0.613675,0.915489,0.014766,0.069746,0.069746,0.014766,0.029532,0.885957,839.168554
4,A,5,-0.389023,3183,786691.62,247.154138,50.0,6.007656,3.398743,3.4,...,1.0,0.613675,0.915489,0.014766,0.069746,0.069746,0.014766,0.029532,0.885957,839.168554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,G,596,-0.007087,14750,3648047.60,247.325261,50.0,9.210035,3.399376,3.4,...,1.0,0.403945,0.889356,0.027458,0.083119,0.083119,0.027458,0.073831,0.815525,840.143094
4196,G,597,-0.007087,14750,3648047.60,247.325261,50.0,9.210035,3.399376,3.4,...,1.0,0.403945,0.889356,0.027458,0.083119,0.083119,0.027458,0.073831,0.815525,840.143094
4197,G,598,-0.007087,14750,3648047.60,247.325261,50.0,9.210035,3.399376,3.4,...,1.0,0.403945,0.889356,0.027458,0.083119,0.083119,0.027458,0.073831,0.815525,840.143094
4198,G,599,-0.007087,14750,3648047.60,247.325261,50.0,9.210035,3.399376,3.4,...,1.0,0.403945,0.889356,0.027458,0.083119,0.083119,0.027458,0.073831,0.815525,840.143094


## Feature Summary and Statistics

In [33]:
# Loan-level feature statistics
print("=== LOAN-LEVEL FEATURE STATISTICS ===")
print(f"Total loans: {len(loan_features_df)}")
print(f"Total features: {len(loan_features_df.columns)}")
print(f"Missing values per feature:")
missing_values = loan_features_df.isnull().sum()
for feature, missing in missing_values[missing_values > 0].items():
    print(f"  {feature}: {missing} ({missing/len(loan_features_df)*100:.1f}%)")

print("\n=== COHORT-LEVEL FEATURE STATISTICS ===")
print(f"Total cohorts: {len(cohort_features_df)}")
print(f"Total features: {len(cohort_features_df.columns)}")
print(f"Missing values per feature:")
missing_values_cohort = cohort_features_df.isnull().sum()
for feature, missing in missing_values_cohort[missing_values_cohort > 0].items():
    print(f"  {feature}: {missing} ({missing/len(cohort_features_df)*100:.1f}%)")

=== LOAN-LEVEL FEATURE STATISTICS ===
Total loans: 36287
Total features: 30
Missing values per feature:
  repayment_velocity_133d: 16356 (45.1%)
  loan_roi_133d: 16356 (45.1%)
  repayment_velocity_266d: 6299 (17.4%)
  loan_roi_266d: 6299 (17.4%)
  repayment_velocity_399d: 47 (0.1%)
  loan_roi_399d: 47 (0.1%)
  days_to_first_repayment: 666 (1.8%)
  repayment_consistency_cv: 666 (1.8%)
  repayment_at_H: 451 (1.2%)

=== COHORT-LEVEL FEATURE STATISTICS ===
Total cohorts: 7
Total features: 30
Missing values per feature:


## Save Features to Database

We'll save both loan-level and cohort-level features to separate tables in the database for easy access in modeling.

In [34]:
# Save features to database
print("Saving features to database...")
save_features_to_database(
    loan_features_df=loan_features_df,
    cohort_features_df=cohort_features_with_labels,
    database_path=DATABASE_PATH,
    decision_time_days=DECISION_TIME_DAYS,
    time_horizon_days=TIME_HORIZON_DAYS
)

print("Features saved successfully!")
print(f"Loan-level features saved to: loan_features_t{DECISION_TIME_DAYS}")
print(f"Cohort-level features saved to: cohort_features_t{DECISION_TIME_DAYS}")

Saving features to database...
Saved 36287 loan features to table: loan_features_t400_h600
Saved 4200 cohort features to table: cohort_features_t400_h600
Features saved successfully!
Loan-level features saved to: loan_features_t400
Cohort-level features saved to: cohort_features_t400


## Next Steps

The feature engineering is complete. Key outputs:

1. **Loan-level features** (`loan_features_t100_h600` table): Individual loan characteristics and early behavior signals
2. **Cohort-level features** (`cohort_features_t100_h600` table): Portfolio composition and risk metrics

### For Modeling:
- **Strategy A (Loan-level → Aggregate)**: Use loan-level features to predict individual outcomes, then aggregate to cohort level
- **Strategy B (Direct Cohort)**: Use cohort-level features to directly predict cohort ROI
